# Gel

> 使用 `gel` 作为后端实现的 LangChain 向量存储抽象。

[Gel](https://www.geldata.com/) 是一个开源的 PostgreSQL 数据层，为从开发到生产的快速周期进行了优化。它提供了一个高级的、严格类型化的类图数据模型、可组合的分层查询语言、完整的 SQL 支持、迁移、Auth 和 AI 模块。

代码位于一个名为 [langchain-gel](https://github.com/geldata/langchain-gel) 的集成包中。

## 设置

首先安装相关包：

In [ ]:
! pip install -qU gel langchain-gel 

## 初始化

为了将 Gel 作为 `VectorStore` 的后端，你需要一个可用的 Gel 实例。
幸运的是，这并不一定需要复杂的 Docker 容器或任何其他东西，除非你愿意！

要设置本地实例，请运行：

In [ ]:
! gel project init --non-interactive

如果你正在使用 [Gel Cloud](https://cloud.geldata.com/)（你应该用！），请在命令中添加一个额外的参数：

```bash
gel project init --server-instance <org-name>/<instance-name>
```

要全面了解 Gel 的运行方式，请参阅参考文档中的 [Running Gel](https://docs.geldata.com/reference/running) 部分。

### 设置 schema

[Gel schema](https://docs.geldata.com/reference/datamodel) 是你应用程序数据模型的显式高级描述。
除了让你精确定义数据布局方式外，它还驱动着 Gel 的许多强大功能，例如 links、access policies、functions、triggers、constraints、indexes 等等。

LangChain 的 `VectorStore` 期望 schema 具有以下布局：

In [14]:
schema_content = """
using extension pgvector;
                                    
module default {
    scalar type EmbeddingVector extending ext::pgvector::vector<1536>;

    type Record {
        required collection: str;
        text: str;
        embedding: EmbeddingVector; 
        external_id: str {
            constraint exclusive;
        };
        metadata: json;

        index ext::pgvector::hnsw_cosine(m := 16, ef_construction := 128)
            on (.embedding)
    } 
}
""".strip()

with open("dbschema/default.gel", "w") as f:
    f.write(schema_content)

为了将 schema 更改应用到数据库，请使用 Gel 的 [migration 机制](https://docs.geldata.com/reference/datamodel/migrations) 来运行迁移：

In [ ]:
! gel migration create --non-interactive
! gel migrate

从这一点开始，`GelVectorStore` 可以作为 LangChain 中任何其他可用向量存储的即插即用替代品。

## 实例化

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>

In [2]:
# | output: false
# | echo: false
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
from langchain_gel import GelVectorStore

vector_store = GelVectorStore(
    embeddings=embeddings,
)

## 管理向量存储

### 向向量存储添加条目

请注意，通过 ID 添加文档将覆盖所有匹配该 ID 的现有文档。

In [ ]:
from langchain_core.documents import Document

docs = [
    Document(
        page_content="there are cats in the pond",
        metadata={"id": "1", "location": "pond", "topic": "animals"},
    ),
    Document(
        page_content="ducks are also found in the pond",
        metadata={"id": "2", "location": "pond", "topic": "animals"},
    ),
    Document(
        page_content="fresh apples are available at the market",
        metadata={"id": "3", "location": "market", "topic": "food"},
    ),
    Document(
        page_content="the market also sells fresh oranges",
        metadata={"id": "4", "location": "market", "topic": "food"},
    ),
    Document(
        page_content="the new art exhibit is fascinating",
        metadata={"id": "5", "location": "museum", "topic": "art"},
    ),
    Document(
        page_content="a sculpture exhibit is also at the museum",
        metadata={"id": "6", "location": "museum", "topic": "art"},
    ),
    Document(
        page_content="a new coffee shop opened on Main Street",
        metadata={"id": "7", "location": "Main Street", "topic": "food"},
    ),
    Document(
        page_content="the book club meets at the library",
        metadata={"id": "8", "location": "library", "topic": "reading"},
    ),
    Document(
        page_content="the library hosts a weekly story time for kids",
        metadata={"id": "9", "location": "library", "topic": "reading"},
    ),
    Document(
        page_content="a cooking class for beginners is offered at the community center",
        metadata={"id": "10", "location": "community center", "topic": "classes"},
    ),
]

vector_store.add_documents(docs, ids=[doc.metadata["id"] for doc in docs])

### 从向量存储中删除项目

In [5]:
vector_store.delete(ids=["3"])

## 查询向量库

在创建了向量库并添加了相关文档后，你很可能希望在运行链或代理时进行查询。

### 过滤支持

向量库支持一组可以应用于文档元数据字段的过滤器。

| 操作符 | 含义/类别               |
|--------|--------------------------|
| \$eq   | 等于 (==)                |
| \$ne   | 不等于 (!=)              |
| \$lt   | 小于 (&lt;)               |
| \$lte  | 小于等于 (&lt;=)          |
| \$gt   | 大于 (>)                 |
| \$gte  | 大于等于 (>=)            |
| \$in   | 特殊情况 (in)            |
| \$nin  | 特殊情况 (not in)        |
| \$between | 特殊情况 (between)       |
| \$like | 文本 (like)              |
| \$ilike | 文本 (不区分大小写的 like) |
| \$and  | 逻辑 (and)               |
| \$or   | 逻辑 (or)                |

### 直接查询

进行简单的相似性搜索可以如下完成：

In [ ]:
results = vector_store.similarity_search(
    "kitty", k=10, filter={"id": {"$in": ["1", "5", "2", "9"]}}
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

如果你提供一个包含多个字段但没有运算符的字典，顶层将被解释为逻辑 **AND** 过滤器

In [ ]:
vector_store.similarity_search(
    "ducks",
    k=10,
    filter={
        "id": {"$in": ["1", "5", "2", "9"]},
        "location": {"$in": ["pond", "market"]},
    },
)

In [ ]:
vector_store.similarity_search(
    "ducks",
    k=10,
    filter={
        "$and": [
            {"id": {"$in": ["1", "5", "2", "9"]}},
            {"location": {"$in": ["pond", "market"]}},
        ]
    },
)

如果您想执行相似性搜索并获得相应的得分，可以运行：

In [ ]:
results = vector_store.similarity_search_with_score(query="cats", k=1)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

### 查询转换为检索器

你也可以将向量存储转换为检索器，以便在你的链中使用。

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
retriever.invoke("kitty")

## 用于检索增强生成 (Retrieval-Augmented Generation) 的用法

有关如何将此向量存储用于检索增强生成 (RAG) 的指南，请参阅以下部分：

- [教程](/docs/tutorials/)
- [操作指南：使用 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval)

## API 参考

如需了解 GelVectorStore 的所有功能和配置的详细文档，请访问 API 参考：https://python.langchain.com/api_reference/